In [ ]:
%load_ext autoreload
%autoreload 2

from collections import namedtuple
from datetime import datetime
import requests
import sqlite3
import sys
import time

from readability import Document
from googlesearch import search

sys.path.insert(0, "..")

from src.vedastro_api import VedAstroApi

# 1 Define astrological chart

In [ ]:
location = "Mumbai"
birth_datetime = "20-03-1996 17:33"
api = VedAstroApi(location, birth_datetime)

In [ ]:
response = api.get_planet_details()
response

In [ ]:
next(k for k in response if "Rahu" in k)["Rahu"]

In [ ]:
Planet = namedtuple("Planet", ["name", "house", "sign", "nakshatra", "combust", "retrograde", "karaka"])
Conjunction = namedtuple("Conjunction", ["planet1", "planet2"])

In [ ]:
chart = [
    Planet("Sun", 8, "Pisces", "Uttara Bhadrapada", False, False, "Putra karaka"),
    Planet("Mercury", 7, "Aquarius", "Purva Bhadrapada", False, False, "Atma karaka"),
    Planet("Venus", 9, "Aries", "Bharani", False, False, "Amatya karaka"),
    Planet("Moon", 8, "Pisces", "Revati", False, False, "Matri karaka"),
    Planet("Mars", 7, "Pisces", "Purva Bhadrapada", True, False, "Dara karaka"),
    Planet("Jupiter", 5, "Sagittarius", "Purva Ashada", False, False, "Bhratri karaka"),
    Planet("Saturn", 8, "Pisces", "Uttara Bhadrapada", True, False, "Gnati karaka"),
    Planet("Rahu", 2, "Virgo", "Chitra", False, True, None),
    Planet("Ketu", 8, "Pisces", "Revati", False, True, None),
]
conjunctions = [
    Conjunction("Mercury", "Mars"),
    Conjunction("Mercury", "Saturn"),
    Conjunction("Mercury", "Sun"),
    Conjunction("Mars", "Saturn"),
    Conjunction("Mars", "Sun"),
    Conjunction("Saturn", "Sun"),
    Conjunction("Moon", "Ketu"),
]

# 2 Retrieve relevant links

In [ ]:
queries = []
for planet in chart:
    queries.append(f"{planet.name} in {planet.house} house")
    queries.append(f"{planet.name} in {planet.sign}")
    queries.append(f"{planet.name} in {planet.nakshatra}")
queries

In [ ]:
# google search
urls = []
pause_per_query = 3
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0'
for query in queries:
	query += " site:astrosaxena.in"
	for j in search(query, tld="co.in", num=3, stop=3, pause=3, user_agent=user_agent):
		urls.append(j)
	time.sleep(pause_per_query)

# 3 Extract content from links

In [ ]:
def get_url_content(url: str):
    response = requests.get(url, headers={'User-Agent': user_agent})
    doc = Document(response.content)
    soup = BeautifulSoup(doc.summary(), 'html.parser')
    text = soup.get_text()
    return doc.title(), text

get_url_content("https://www.astrosaxena.in/astrology/info/703/concept-of-combust-planets")

In [ ]:
create_table_sql = """
create table if not exists knowledge_base (
    id INTEGER primary key autoincrement,
    url TEXT not null unique,
    content TEXT not null,
    created_at TIMESTAMP not null default CURRENT_TIMESTAMP,
);
"""
connection = sqlite3.connect("astrosaxena.db")
cursor = connection.cursor()
cursor.execute(create_table_sql)
try:
    for url in urls:
        text = get_url_content(url)
        cursor.execute("insert into knowledge_base values (url, content)")
        connection.commit()
finally:
    connection.close()

# 4 Langchain